In [3]:
# CAMERA CALIBRATION dataset1

import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
img1 = cv2.imread("im0.png")
img2 = cv2.imread("im1.png")
img1 = cv2.resize(img1,(1000,700))
img2 = cv2.resize(img2,(1000,700))

k1 = np.array([[5299.313, 0, 1432.004],[0, 5299.313, 977.763],[0, 0, 1]])
k2 = np.array([[5299.313, 0, 1438.004],[0, 5299.313, 977.763],[0, 0, 1]])

def normalised_mat(features):
    mean_features = np.mean(features, axis =0)
    meanx = mean_features[0]
    meany = mean_features[1]
    feature_x = features[:,0] - meanx
    feature_y = features[:,1] - meany
    s = (np.sqrt(2)/np.mean(feature_x**2 + feature_y**2))**(1/2)
    #s = 2 / (np.mean(np.sum(feature_x ** 2), (feature_y ** 2)) ** (1 /2))
    a1 = np.array([[s, 0 ,0],
                   [0, s, 0],
                   [0, 0, 1]])
    a2 = np.array([[1, 0, -(meanx)], [0, 1, -(meany)], [0, 0, 1]])
    a = a1 .dot(a2)
    ones = np.ones(len(features))
    n = np.column_stack((features,np.ones(len(features))))
    n_normalised = ((a.dot(n.T).T))
    return n_normalised , a

def normalized_fund_mat(matches,n = True):
    x1 = matches[:,0:2]
    x2 = matches[:,2:4]
    if x1.shape[0] > 7:
        if n == True:
            n1_normalised, a11 = normalised_mat(x1)
            n2_normalised, a22 = normalised_mat(x2)
        else:
            n1_normalised = x1
            n2_normalised = x2
        matrix_A = np.zeros((len(n1_normalised),9))
        for i in range(0, len(n1_normalised)):
            x1= n1_normalised[i][0]
            y1= n1_normalised[i][1]
            x2= n1_normalised[i][0]
            y2= n1_normalised[i][1]
            matrix_A[i] = np.array([x1*x2, x2*y1, x2, y2*x1, y1*y2, y2,x1, y1,1])

        U, S, V = np.linalg.svd(matrix_A)
        F_mat  = V.T[:, -1]
        F_mat  = F_mat.reshape(3,3)
        u, s, vt = np.linalg.svd(F_mat)
        s = np.diag(s)
        s[2,2] = 0
        F_mat = np.dot(u, np.dot(s, vt))
        if n:
            normalised_F = np.dot(a22.T ,np.dot(F_mat ,a11))
        return normalised_F

def errorF(features, f):
    f1 = features[0:2]
    f2 = features[2:4]
    error1 = np.array([f1[0], f1[1], 1]).T
    error2 = np.array([f2[0], f2[1], 1])
    error = np.dot(error1, np.dot(f, error2))
    abserr = np.abs(error)
    return abserr

def ransac(features):
    f_in = []
    f_best = 0
    N = 1000
    e = 0.05
    in_c = 0
    for i in range(0,N):
        ind = []
        f1 = features.shape[0]
        random_list = np.random.choice(f1, size = 8)
        feat_rand8 = features[random_list, :]
        fm = normalized_fund_mat(feat_rand8)
        for m in range(f1):
            f2 = features[m]
            err = errorF(f2, fm)
            if err < e:
                ind.append(m)
        if len(ind) > in_c:
            in_c = len(ind)
            f_best = fm
            f_in = ind
    final_features = features[f_in, :]
    return f_best, final_features
def essential_matrix(f , k1, k2):
    E = k2.T.dot(f).dot(k1)
    U,S,V = np.linalg.svd(E)
    S[0]=1
    S[1]=1
    S[2]=0
    SS = np.diag(S)
    EE = np.dot(U,np.dot(SS,V))
    return EE
def camera_pos(essen_mat):
    U_dec, S_dec, V_dec = np.linalg.svd(essen_mat)
    W= np.array([[0,-1,0],[1,0,0],[0,0,1]])
    R = []
    C = []
    c1 = U_dec[:,2]
    c2 = -U_dec[:,2]
    c3 = U_dec[:,2]
    c4 = -U_dec[:,2]
    R1 = U_dec @ W @ V_dec
    R2 = U_dec @ W @ V_dec
    R3 = U_dec @ (W.T) @ V_dec
    R4 = U_dec @ (W.T) @ V_dec
    if (np.linalg.det(R1)) < 0:
        c1 = -c1
        R1 = -R1
    if (np.linalg.det(R2)) < 0:
        c2 = -c2
        R2 = -R2
    if (np.linalg.det(R3)) <0:
        c3 = -c3
        R3 = -R3
    if (np.linalg.det(R4)) < 0:
        c4 = -c4
        R4 = -R4
    c1 = c1.reshape(3,1)
    c2 = c2.reshape(3,1)
    c3 = c3.reshape(3,1)
    c4 = c4.reshape(3,1)
    return [c1,c2,c3,c4],[R1,R2,R3,R4]
orb = cv2.xfeatures2d.SIFT_create()

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x :x.distance)
good = matches[0:90]
matched = []
for i,m in enumerate(good):
    feature1 = (kp1[m.queryIdx].pt)
    feature2 = (kp2[m.trainIdx].pt)
    matched.append([feature1[0], feature1[1], feature2[0], feature2[1]])
matched = np.array(matched).reshape(-1,4)
F_best, final_features = ransac(matched)
E = essential_matrix(F_best, k1, k2)
C,R = camera_pos(E)
print("C =", C[0])
print("R =", R[0])

C = [[ 9.98710895e-01]
 [-7.29177177e-04]
 [-5.07544788e-02]]
R = [[ 9.94828619e-01 -1.43663136e-03 -1.01557640e-01]
 [-1.44985899e-03 -9.99998947e-01 -5.64346610e-05]
 [-1.01557452e-01  2.03387073e-04 -9.94829655e-01]]


In [4]:
# CAMERA CALIBRATION dataset2

import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
img1 = cv2.imread("data2img1.png")
img2 = cv2.imread("data2img2.png")
img1 = cv2.resize(img1,(1000,700))
img2 = cv2.resize(img2,(1000,700))

k1 = np.array([[5299.313, 0, 1432.004],[0, 5299.313, 977.763],[0, 0, 1]])
k2 = np.array([[5299.313, 0, 1438.004],[0, 5299.313, 977.763],[0, 0, 1]])

def normalised_mat(features):
    mean_features = np.mean(features, axis =0)
    meanx = mean_features[0]
    meany = mean_features[1]
    feature_x = features[:,0] - meanx
    feature_y = features[:,1] - meany
    s = (np.sqrt(2)/np.mean(feature_x**2 + feature_y**2))**(1/2)
    #s = 2 / (np.mean(np.sum(feature_x ** 2), (feature_y ** 2)) ** (1 /2))
    a1 = np.array([[s, 0 ,0],
                   [0, s, 0],
                   [0, 0, 1]])
    a2 = np.array([[1, 0, -(meanx)], [0, 1, -(meany)], [0, 0, 1]])
    a = a1 .dot(a2)
    ones = np.ones(len(features))
    n = np.column_stack((features,np.ones(len(features))))
    n_normalised = ((a.dot(n.T).T))
    return n_normalised , a

def normalized_fund_mat(matches,n = True):
    x1 = matches[:,0:2]
    x2 = matches[:,2:4]
    if x1.shape[0] > 7:
        if n == True:
            n1_normalised, a11 = normalised_mat(x1)
            n2_normalised, a22 = normalised_mat(x2)
        else:
            n1_normalised = x1
            n2_normalised = x2
        matrix_A = np.zeros((len(n1_normalised),9))
        for i in range(0, len(n1_normalised)):
            x1= n1_normalised[i][0]
            y1= n1_normalised[i][1]
            x2= n1_normalised[i][0]
            y2= n1_normalised[i][1]
            matrix_A[i] = np.array([x1*x2, x2*y1, x2, y2*x1, y1*y2, y2,x1, y1,1])

        U, S, V = np.linalg.svd(matrix_A)
        F_mat  = V.T[:, -1]
        F_mat  = F_mat.reshape(3,3)
        u, s, vt = np.linalg.svd(F_mat)
        s = np.diag(s)
        s[2,2] = 0
        F_mat = np.dot(u, np.dot(s, vt))
        if n:
            normalised_F = np.dot(a22.T ,np.dot(F_mat ,a11))
        return normalised_F

def errorF(features, f):
    f1 = features[0:2]
    f2 = features[2:4]
    error1 = np.array([f1[0], f1[1], 1]).T
    error2 = np.array([f2[0], f2[1], 1])
    error = np.dot(error1, np.dot(f, error2))
    abserr = np.abs(error)
    return abserr

def ransac(features):
    f_in = []
    f_best = 0
    N = 1000
    e = 0.05
    in_c = 0
    for i in range(0,N):
        ind = []
        f1 = features.shape[0]
        random_list = np.random.choice(f1, size = 8)
        feat_rand8 = features[random_list, :]
        fm = normalized_fund_mat(feat_rand8)
        for m in range(f1):
            f2 = features[m]
            err = errorF(f2, fm)
            if err < e:
                ind.append(m)
        if len(ind) > in_c:
            in_c = len(ind)
            f_best = fm
            f_in = ind
    final_features = features[f_in, :]
    return f_best, final_features
def essential_matrix(f , k1, k2):
    E = k2.T.dot(f).dot(k1)
    U,S,V = np.linalg.svd(E)
    S[0]=1
    S[1]=1
    S[2]=0
    SS = np.diag(S)
    EE = np.dot(U,np.dot(SS,V))
    return EE
def camera_pos(essen_mat):
    U_dec, S_dec, V_dec = np.linalg.svd(essen_mat)
    W= np.array([[0,-1,0],[1,0,0],[0,0,1]])
    R = []
    C = []
    c1 = U_dec[:,2]
    c2 = -U_dec[:,2]
    c3 = U_dec[:,2]
    c4 = -U_dec[:,2]
    R1 = U_dec @ W @ V_dec
    R2 = U_dec @ W @ V_dec
    R3 = U_dec @ (W.T) @ V_dec
    R4 = U_dec @ (W.T) @ V_dec
    if (np.linalg.det(R1)) < 0:
        c1 = -c1
        R1 = -R1
    if (np.linalg.det(R2)) < 0:
        c2 = -c2
        R2 = -R2
    if (np.linalg.det(R3)) <0:
        c3 = -c3
        R3 = -R3
    if (np.linalg.det(R4)) < 0:
        c4 = -c4
        R4 = -R4
    c1 = c1.reshape(3,1)
    c2 = c2.reshape(3,1)
    c3 = c3.reshape(3,1)
    c4 = c4.reshape(3,1)
    return [c1,c2,c3,c4],[R1,R2,R3,R4]
orb = cv2.xfeatures2d.SIFT_create()

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x :x.distance)
good = matches[0:90]
matched = []
for i,m in enumerate(good):
    feature1 = (kp1[m.queryIdx].pt)
    feature2 = (kp2[m.trainIdx].pt)
    matched.append([feature1[0], feature1[1], feature2[0], feature2[1]])
matched = np.array(matched).reshape(-1,4)
F_best, final_features = ransac(matched)
E = essential_matrix(F_best, k1, k2)
C,R = camera_pos(E)
print("C =", C[0])
print("R =", R[0])

C = [[ 0.99960946]
 [ 0.00341481]
 [-0.02773564]]
R = [[ 9.98460435e-01  6.76982438e-03 -5.50538751e-02]
 [ 6.80786991e-03 -9.99976699e-01  5.03545072e-04]
 [-5.50491834e-02 -8.77569451e-04 -9.98483258e-01]]


In [5]:
# CAMERA CALIBRATION dataset3

import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
img1 = cv2.imread("data3img1.png")
img2 = cv2.imread("data3img2.png")
img1 = cv2.resize(img1,(1000,700))
img2 = cv2.resize(img2,(1000,700))

k1 = np.array([[5299.313, 0, 1432.004],[0, 5299.313, 977.763],[0, 0, 1]])
k2 = np.array([[5299.313, 0, 1438.004],[0, 5299.313, 977.763],[0, 0, 1]])

def normalised_mat(features):
    mean_features = np.mean(features, axis =0)
    meanx = mean_features[0]
    meany = mean_features[1]
    feature_x = features[:,0] - meanx
    feature_y = features[:,1] - meany
    s = (np.sqrt(2)/np.mean(feature_x**2 + feature_y**2))**(1/2)
    #s = 2 / (np.mean(np.sum(feature_x ** 2), (feature_y ** 2)) ** (1 /2))
    a1 = np.array([[s, 0 ,0],
                   [0, s, 0],
                   [0, 0, 1]])
    a2 = np.array([[1, 0, -(meanx)], [0, 1, -(meany)], [0, 0, 1]])
    a = a1 .dot(a2)
    ones = np.ones(len(features))
    n = np.column_stack((features,np.ones(len(features))))
    n_normalised = ((a.dot(n.T).T))
    return n_normalised , a

def normalized_fund_mat(matches,n = True):
    x1 = matches[:,0:2]
    x2 = matches[:,2:4]
    if x1.shape[0] > 7:
        if n == True:
            n1_normalised, a11 = normalised_mat(x1)
            n2_normalised, a22 = normalised_mat(x2)
        else:
            n1_normalised = x1
            n2_normalised = x2
        matrix_A = np.zeros((len(n1_normalised),9))
        for i in range(0, len(n1_normalised)):
            x1= n1_normalised[i][0]
            y1= n1_normalised[i][1]
            x2= n1_normalised[i][0]
            y2= n1_normalised[i][1]
            matrix_A[i] = np.array([x1*x2, x2*y1, x2, y2*x1, y1*y2, y2,x1, y1,1])

        U, S, V = np.linalg.svd(matrix_A)
        F_mat  = V.T[:, -1]
        F_mat  = F_mat.reshape(3,3)
        u, s, vt = np.linalg.svd(F_mat)
        s = np.diag(s)
        s[2,2] = 0
        F_mat = np.dot(u, np.dot(s, vt))
        if n:
            normalised_F = np.dot(a22.T ,np.dot(F_mat ,a11))
        return normalised_F

def errorF(features, f):
    f1 = features[0:2]
    f2 = features[2:4]
    error1 = np.array([f1[0], f1[1], 1]).T
    error2 = np.array([f2[0], f2[1], 1])
    error = np.dot(error1, np.dot(f, error2))
    abserr = np.abs(error)
    return abserr

def ransac(features):
    f_in = []
    f_best = 0
    N = 1000
    e = 0.05
    in_c = 0
    for i in range(0,N):
        ind = []
        f1 = features.shape[0]
        random_list = np.random.choice(f1, size = 8)
        feat_rand8 = features[random_list, :]
        fm = normalized_fund_mat(feat_rand8)
        for m in range(f1):
            f2 = features[m]
            err = errorF(f2, fm)
            if err < e:
                ind.append(m)
        if len(ind) > in_c:
            in_c = len(ind)
            f_best = fm
            f_in = ind
    final_features = features[f_in, :]
    return f_best, final_features
def essential_matrix(f , k1, k2):
    E = k2.T.dot(f).dot(k1)
    U,S,V = np.linalg.svd(E)
    S[0]=1
    S[1]=1
    S[2]=0
    SS = np.diag(S)
    EE = np.dot(U,np.dot(SS,V))
    return EE
def camera_pos(essen_mat):
    U_dec, S_dec, V_dec = np.linalg.svd(essen_mat)
    W= np.array([[0,-1,0],[1,0,0],[0,0,1]])
    R = []
    C = []
    c1 = U_dec[:,2]
    c2 = -U_dec[:,2]
    c3 = U_dec[:,2]
    c4 = -U_dec[:,2]
    R1 = U_dec @ W @ V_dec
    R2 = U_dec @ W @ V_dec
    R3 = U_dec @ (W.T) @ V_dec
    R4 = U_dec @ (W.T) @ V_dec
    if (np.linalg.det(R1)) < 0:
        c1 = -c1
        R1 = -R1
    if (np.linalg.det(R2)) < 0:
        c2 = -c2
        R2 = -R2
    if (np.linalg.det(R3)) <0:
        c3 = -c3
        R3 = -R3
    if (np.linalg.det(R4)) < 0:
        c4 = -c4
        R4 = -R4
    c1 = c1.reshape(3,1)
    c2 = c2.reshape(3,1)
    c3 = c3.reshape(3,1)
    c4 = c4.reshape(3,1)
    return [c1,c2,c3,c4],[R1,R2,R3,R4]
orb = cv2.xfeatures2d.SIFT_create()

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x :x.distance)
good = matches[0:90]
matched = []
for i,m in enumerate(good):
    feature1 = (kp1[m.queryIdx].pt)
    feature2 = (kp2[m.trainIdx].pt)
    matched.append([feature1[0], feature1[1], feature2[0], feature2[1]])
matched = np.array(matched).reshape(-1,4)
F_best, final_features = ransac(matched)
E = essential_matrix(F_best, k1, k2)
C,R = camera_pos(E)
print("C =", C[0])
print("R =", R[0])

C = [[ 0.43727702]
 [ 0.09279548]
 [-0.89452658]]
R = [[-0.62555809  0.08110136 -0.7759508 ]
 [ 0.07999924 -0.98267175 -0.16720156]
 [-0.7760652  -0.16666976  0.60823021]]
